# Getting in through the login page

In [447]:
# Download needed packages
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import config

In [ ]:
# Retriving username and password from alternate file
bardough_username = config.username
bardough_password = config.password

In [448]:
# Open the web page that we want open and log in.
path = '/Users/sigfus/Desktop/Lífið/Github/Bardough/Crawling/chromedriver-mac-x64/chromedriver'
s = Service(path)
driver = webdriver.Chrome(service = s)
driver.get('https://www.toasttab.com/login')

In [449]:
# Locate the username input field
username = driver.find_element(By.ID, 'username') 

In [450]:
# Sign in with email first
userid = bardough_username
username.send_keys(userid)

In [451]:
# Click the sign in button to prompt password
sign_in_button = driver.find_element('xpath', '/html/body/div[2]/main/section/div/div/div/div/div/form/div[2]/button')
sign_in_button.click()

In [452]:
# Locate the password input field
password = driver.find_element(By.ID, 'password')

In [453]:
# Sign in with password second
key = bardough_password
password.send_keys(key)

In [454]:
# Click the sign in button to enter page
sign_in_button = driver.find_element('xpath', '/html/body/div[2]/main/section/div/div/div/form/div[3]/button')
sign_in_button.click()

In [455]:
# Get url of Item Detail page manually (under reports --> menu)
url = 'https://www.toasttab.com/restaurants/admin/reports/home#selection-details'
driver.get(url)

# Crawling Item Details data for December 2022 - November 2023

## Code for crawling 2023 item details data

### Month changed for each iteration and total combined at end

In [361]:
# Crawl item details month by month

nov_order_number = []
nov_sent_date = []
nov_menu_item = []
nov_menu_group = []
nov_menu = []
nov_sales_category = []
nov_net_price = []
nov_quantity = []
nov_void = []

page = 0
# Change page count based on number of pages
while page < 69:
    
    # Scan page
    soup = bs(driver.page_source)
    
    # Crawl page
    ## Odd rows
    odd_rows = soup.find_all(class_ = 'odd')
    odd_list = []
    for row in odd_rows:
        # Find all cells in the row and loop through them
        odd_cells = row.find_all('td')
        for cell in odd_cells:
            # Extract text from each cell and convert to int if possible
            odd_text = cell.get_text()
            odd_list.append(odd_text)
    ## Even rows
    even_rows = soup.find_all(class_ = 'even')
    even_list = []
    for row in even_rows:
        # Find all cells in the row and loop through them
        even_cells = row.find_all('td')
        for cell in even_cells:
            # Extract text from each cell and convert to int if possible
            even_text = cell.get_text()
            even_list.append(even_text)
    # Combine lists        
    total_list = odd_list + even_list
    
    # Sort and append lists
    order_number = total_list[::9]
    nov_order_number.append(order_number)
    
    sent_date = total_list[1::9]
    nov_sent_date.append(sent_date)
    
    menu_item = total_list[2::9]
    nov_menu_item.append(menu_item)
    
    menu_group = total_list[3::9]
    nov_menu_group.append(menu_group)
    
    menu = total_list[4::9]
    nov_menu.append(menu)
    
    sales_category = total_list[5::9]
    nov_sales_category.append(sales_category)
    
    net_price = total_list[6::9]
    nov_net_price.append(net_price)
    
    quantity = total_list[7::9]
    nov_quantity.append(quantity)
    
    void = total_list[8::9]
    nov_void.append(void)
    
    # Click to next page
    time.sleep(5)
    #Scroll down to the bottom of the page 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #wait for page to be loaded
    time.sleep(5)
    #find and click 'next' button
    next_page_button = driver.find_element('xpath', '//*[@id="menu-items-report_wrapper"]/div[4]/div/div/ul/li[7]/a')
    next_page_button.click()
    time.sleep(5)
    
    page += 1

In [473]:
# Check for duplicate pages (error in crawling/page switching)
# For loop to select page's list
for i in range(len(nov_order_number)):
    # Nested for loop to go through other pages' lists
    for j in range(i+1, len(nov_order_number)):
        # Evaluation to see if page i's data is equal to other lists' data
        if nov_order_number[i] == nov_order_number[j]:
            # Show where the duplicates are found
            print("The lists at index {} and {} are equal: {}".format(i, j)

In [362]:
# Combine page lists into one huge list
complete_nov_order_number = [inner_item for outer_item in nov_order_number for inner_item in outer_item]
complete_nov_sent_date = [inner_item for outer_item in nov_sent_date for inner_item in outer_item]
complete_nov_menu_item = [inner_item for outer_item in nov_menu_item for inner_item in outer_item]
complete_nov_menu_group = [inner_item for outer_item in nov_menu_group for inner_item in outer_item]
complete_nov_menu = [inner_item for outer_item in nov_menu for inner_item in outer_item]
complete_nov_sales_category = [inner_item for outer_item in nov_sales_category for inner_item in outer_item]
complete_nov_net_price = [inner_item for outer_item in nov_net_price for inner_item in outer_item]
complete_nov_quantity = [inner_item for outer_item in nov_quantity for inner_item in outer_item]
complete_nov_void = [inner_item for outer_item in nov_void for inner_item in outer_item]

In [363]:
# Create data frame for month
nov_item_details = {'Order Number': complete_nov_order_number, 
                'Sent Date': complete_nov_sent_date, 
                'Menu Item': complete_nov_menu_item, 
                'Menu Group': complete_nov_menu_group, 
                'Menu': complete_nov_menu, 
                'Sales Category': complete_nov_sales_category, 
                'Net Price': complete_nov_net_price,
                'Quantity': complete_nov_quantity, 
                'Void?': complete_nov_void
                }

nov_item_details = pd.DataFrame(nov_item_details)
nov_item_details.sort_values(by = "Sent Date")
nov_item_details.shape

(6815, 9)

In [366]:
# Download to CSV
nov_item_details.to_csv('/Users/sigfus/Desktop/nov_item_details.csv', sep = ',', encoding = 'utf-8')

## Combining monthly data frames into one

In [493]:
# Create a list of the 12 monthly data frames
item_details_seperate = [dec_22_item_details, jan_item_details, feb_item_details, 
                         mar_item_details, apr_item_details, may_item_details, 
                         jun_item_details, jul_item_details, aug_item_details, 
                         sep_item_details, oct_item_details, nov_item_details]

In [497]:
# Combine them into one data frame with a new index rather than index from monthly data frames
item_details_full = pd.concat(item_details_seperate, ignore_index=True)
item_details_full.shape

(67459, 9)

In [496]:
# Download to CSV
item_details_full.to_csv('/Users/sigfus/Desktop/item_details_full.csv', sep = ',', encoding = 'utf-8')